In [1]:
import pandas as pd
import pickle 

# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_model_on_k_fold_validation import load_configuration

Training and Hyper-parameter tuning with Ray is not possible


## Load 'args' from a trial id

In [3]:
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_14_16_03_71730'
# 'subway_in_subway_out_STGCN_MSELoss_2025_02_14_16_03_71730' # NEW RIM ARCHITECTURE
# "subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963"
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# "subway_in_STGCN_MSELoss_2025_01_06_08_00_94523"
args,_ = load_configuration(trial_id,load_config=True)

print('Best args: ')
print(args)


>>>> Load best CONFIG
Best args: 
Namespace(model_name='STGCN', dataset_names=['subway_in', 'subway_out'], dataset_for_coverage=['subway_in', 'netmob_POIs'], device=device(type='cuda'), optimizer='adamw', single_station=False, loss_function_type='MSE', epsilon_clustering=0.05, freq='15min', contextual_positions={}, quick_vision=False, netmob_transfer_mode='DL', evaluate_complete_ds=True, train_valid_test_split_method='similar_length_method', set_spatial_units=None, hp_tuning_on_first_fold=True, keep_best_weights=False, num_workers=0, persistent_workers=False, pin_memory=True, prefetch_factor=2, drop_last=False, mixed_precision=False, non_blocking=True, torch_compile=False, backend='inductor', prefetch_all=False, NetMob_selected_apps=['Google_Maps', 'Deezer', 'Instagram'], NetMob_transfer_mode=['DL'], NetMob_selected_tags=['iris'], NetMob_expanded='', ray=False, ray_scheduler='ASHA', ray_search_alg=None, grace_period=20, HP_max_epochs=100, alpha=None, conformity_scores_type='max_residu

In [4]:
df_hp_tuning =pd.read_csv(f'save/HyperparameterTuning/{trial_id}.csv')
model_args = pickle.load(open(f'save/HyperparameterTuning/model_args.pkl','rb'))
metric = '_metric/Loss_model'
# Load common args through all tuning trials:
args = model_args['model'][trial_id]['args']

# Rename columns, remove useless ones: 
columns = [c for c in df_hp_tuning.columns if ('_metric/' in c) or ('config/' in c)]
columns_rename = {c:c.split('/')[-1] for c in columns}
df_best_configs = df_hp_tuning[columns].sort_values(metric).rename(columns = columns_rename)
df_best_configs
print('Best configs:')
display(df_best_configs.iloc[:20])

Best configs:


,Loss_model,lr,weight_decay,dropout,temporal_h_dim,spatial_h_dim,output_h_dim,TGE_num_layers,TGE_num_heads,TGE_FC_hdim,scheduler,torch_scheduler_milestone,torch_scheduler_gamma,torch_scheduler_lr_start_factor
43,0.003555,0.00145,0.093438,0.017829,128,32,64,3,1,8,True,29.0,0.988059,0.600637
180,0.003558,0.00290,0.056935,0.205239,8,128,32,8,2,16,True,16.0,0.994082,0.494039
415,0.003686,0.00310,0.076135,0.209686,128,32,16,4,1,64,True,6.0,0.989726,0.609100
479,0.003715,0.00260,0.015236,0.153810,64,16,256,8,8,16,True,21.0,0.986840,0.273233
72,0.003723,0.00265,0.056675,0.087503,64,128,64,1,1,8,True,13.0,0.987510,0.355194
488,0.003726,0.00030,0.035155,0.013960,32,32,256,1,1,256,NaN,NaN,NaN,NaN
234,0.003740,0.00060,0.082800,0.119636,32,256,256,2,8,16,True,14.0,0.991411,0.928782
262,0.003776,0.00125,0.061103,0.065937,16,64,32,3,8,64,True,13.0,0.989121,0.202832
247,0.003781,0.00245,0.054753,0.018427,32,128,8,4,1,64,NaN,NaN,NaN,NaN
57,0.003792,0.00015,0.068777,0.034193,128,8,256,8,2,256,True,6.0,0.996067,0.284229


## Load best configs from a trial id:

In [3]:
df_hp_tuning =pd.read_csv(f'save/HyperparameterTuning/{trial_id}.csv')
model_args = pickle.load(open(f'save/HyperparameterTuning/model_args.pkl','rb'))
metric = '_metric/Loss_model'
# Load common args through all tuning trials:
args = model_args['model'][trial_id]['args']

# Rename columns, remove useless ones: 
columns = [c for c in df_hp_tuning.columns if ('_metric/' in c) or ('config/' in c)]
columns_rename = {c:c.split('/')[-1] for c in columns}
df_best_configs = df_hp_tuning[columns].sort_values(metric).rename(columns = columns_rename)
df_best_configs
print('Best configs:')
display(df_best_configs.iloc[:20])

Best configs:


,Loss_model,lr,weight_decay,dropout,temporal_h_dim,spatial_h_dim,output_h_dim,scheduler,torch_scheduler_milestone,torch_scheduler_gamma,torch_scheduler_lr_start_factor,concatenation_early,concatenation_late,num_heads,grn_out_dim
351,0.005105,0.00065,0.007052,0.278061,256,256,8,True,25.0,0.991593,0.752912,True,False,2,64
226,0.005124,0.00115,0.065237,0.159976,16,8,8,NaN,NaN,NaN,NaN,True,True,4,32
414,0.005195,0.00055,0.024583,0.134271,32,256,16,NaN,NaN,NaN,NaN,True,True,4,16
316,0.005217,0.00110,0.022065,0.100491,256,64,32,True,28.0,0.995704,0.542937,True,True,1,16
353,0.005256,0.00400,0.072955,0.068490,32,8,8,NaN,NaN,NaN,NaN,True,False,3,24
358,0.005258,0.00260,0.017781,0.522119,16,8,256,NaN,NaN,NaN,NaN,True,False,1,8
255,0.005270,0.00055,0.065677,0.287299,256,256,128,NaN,NaN,NaN,NaN,True,False,2,8
272,0.005285,0.00485,0.015913,0.078816,64,64,256,True,2.0,0.998806,0.508260,True,False,2,32
286,0.005343,0.00070,0.033701,0.385530,256,64,128,True,7.0,0.990168,0.132749,True,True,6,48
341,0.005344,0.00390,0.037983,0.313570,128,16,32,NaN,NaN,NaN,NaN,True,True,3,24


## Load `the` best config, with no rename: 

In [8]:
# Get best config :
best_model = df_hp_tuning.sort_values(metric).iloc[0]
print('raw config of best model: ')
display(best_model)

raw config of best model: 


trial_id                                                                                       b9f67_00351
time_this_iter_s                                                                                  4.243078
done                                                                                                  True
training_iteration                                                                                     100
date                                                                                   2025-01-07_01-20-56
timestamp                                                                                       1736209256
time_total_s                                                                                    442.195914
pid                                                                                                 144430
hostname                                                                                 ticil2.ifsttar.fr
node_ip                              